<a href="https://colab.research.google.com/github/AgrawalHimanshi/ML-NLP/blob/master/flight%20price%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import seaborn as sns
import numpy as np

In [7]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Data_Train.xlsx to Data_Train.xlsx
User uploaded file "Data_Train.xlsx" with length 530389 bytes


In [8]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Test_set.xlsx to Test_set.xlsx
User uploaded file "Test_set.xlsx" with length 120774 bytes


In [0]:
df_train = pd.read_excel("Data_Train.xlsx")
df_test=pd.read_excel("Test_set.xlsx")

In [10]:
print("Number of rows in training dataset",df_train.shape)
print("Number of rows in prediction dataset",df_test.shape)

Number of rows in training dataset (10683, 11)
Number of rows in prediction dataset (2671, 10)


In [11]:
df_train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [12]:
#check for null values
df_train.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [14]:
#check null columns
null_col=df_train.columns[df_train.isnull().any()]
print(df_train[df_train.isnull().any(axis=1)][null_col].head())

     Route Total_Stops
9039   NaN         NaN


In [0]:
#drop null col values
df_train.dropna(inplace=True)

In [16]:
#select duplicated rows except first occurrence in all columns
duplicaterows=df_train[df_train.duplicated()]
print("duplicated rows except first occurrence in all columns",duplicaterows.shape)

duplicated rows except first occurrence in all columns (220, 11)


In [0]:
#remove duplicate rows
df_train.drop_duplicates(keep='first',inplace=True)

In [18]:

df_train.shape

(10462, 11)

In [19]:
print("Train Set:\n",df_train["Additional_Info"].value_counts())
print("\nPrediction Set:\n",df_test["Additional_Info"].value_counts())





Train Set:
 No info                         8182
In-flight meal not included     1926
No check-in baggage included     318
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
1 Short layover                    1
Red-eye flight                     1
2 Long layover                     1
Name: Additional_Info, dtype: int64

Prediction Set:
 No info                         2148
In-flight meal not included      444
No check-in baggage included      76
Business class                     1
Change airports                    1
1 Long layover                     1
Name: Additional_Info, dtype: int64


In [0]:

# replace repeating value in train set
df_train["Additional_Info"]=df_train["Additional_Info"].replace({'No Info':'No info'})



In [0]:
df_train.Total_Stops.replace(['1 stop', 'non-stop', '2 stops', '3 stops', '4 stops'], [1, 0, 2, 3, 4], inplace=True)
df_train['Total_Stops']


In [0]:
df_test['Total_Stops'].replace(['1 stop', 'non-stop', '2 stops', '3 stops', '4 stops'], [1, 0, 2, 3, 4], inplace=True)

In [23]:
df_train["Total_Stops"].dtype

dtype('int64')

In [0]:
#feature engineering on date of journey column
df_train["isWeekend"] = ((pd.to_datetime(df_train["Date_of_Journey"], format = '%d/%m/%Y').dt.dayofweek) // 5 == 1).astype(int)

In [0]:
df_train["Day_of_week"] = (pd.to_datetime(df_train["Date_of_Journey"], format = '%d/%m/%Y').dt.day_name()) 
df_train["Day_of_journey"] = (pd.to_datetime(df_train["Date_of_Journey"], format = '%d/%m/%Y').dt.day)
df_train["Month_of_journey"] = (pd.to_datetime(df_train["Date_of_Journey"], format = '%d/%m/%Y').dt.month) 

In [0]:
df_test["isWeekend"] = ((pd.to_datetime(df_test["Date_of_Journey"], format = '%d/%m/%Y').dt.dayofweek) // 5 == 1).astype(int)
df_test["Day_of_week"] = (pd.to_datetime(df_test["Date_of_Journey"], format = '%d/%m/%Y').dt.day_name()) 
df_test["Day_of_journey"] = (pd.to_datetime(df_test["Date_of_Journey"], format = '%d/%m/%Y').dt.day)
df_test["Month_of_journey"] = (pd.to_datetime(df_test["Date_of_Journey"], format = '%d/%m/%Y').dt.month) 

In [0]:
#drop original column from train and test
df_train.drop(labels = 'Date_of_Journey', axis = 1, inplace = True)


In [0]:
df_test.drop(labels='Date_of_Journey', axis=1, inplace=True)

In [0]:
#converting duration into min
duration = list(df_train["Duration"])

#separating min and hours
for i in range(len(duration)) :
    if len(duration[i].split()) != 2:
        if 'h' in duration[i] :
            duration[i] = duration[i].strip() + ' 0m'
        elif 'm' in duration[i] :
            duration[i] = '0h {}'.format(duration[i].strip())

dur_hours = []
dur_minutes = []  
dur_seconds = []

#converting into minutes
for i in range(len(duration)) :
    dur_minutes.append(int(duration[i].split()[0][:-1])*60 + int(duration[i].split()[1][:-1]))
    dur_seconds.append(int(duration[i].split()[0][:-1])*60*60 + int(duration[i].split()[1][:-1])*60)
df_train["Duration_minutes"] = dur_minutes
df_train.drop(["Duration"], axis=1, inplace = True)

In [0]:
#test data
#converting duration into min
duration = list(df_test["Duration"])

#separating min and hours
for i in range(len(duration)) :
    if len(duration[i].split()) != 2:
        if 'h' in duration[i] :
            duration[i] = duration[i].strip() + ' 0m'
        elif 'm' in duration[i] :
            duration[i] = '0h {}'.format(duration[i].strip())

dur_hours = []
dur_minutes = []  
dur_seconds = []

#converting into minutes
for i in range(len(duration)) :
    dur_minutes.append(int(duration[i].split()[0][:-1])*60 + int(duration[i].split()[1][:-1]))
    dur_seconds.append(int(duration[i].split()[0][:-1])*60*60 + int(duration[i].split()[1][:-1])*60)
df_test["Duration_minutes"] = dur_minutes
df_test.drop(["Duration"], axis=1, inplace = True)

In [0]:
#Cleaning Departure and Arrival Times
# Training Set
df_train['Depart_Time_Hour'] = pd.to_datetime(df_train.Dep_Time).dt.hour
df_train['Depart_Time_Minutes'] = pd.to_datetime(df_train.Dep_Time).dt.minute
df_train.drop(labels = 'Dep_Time', axis = 1, inplace = True)
df_train['Arr_Time_Hour'] = pd.to_datetime(df_train.Arrival_Time).dt.hour
df_train['Arr_Time_Minutes'] = pd.to_datetime(df_train.Arrival_Time).dt.minute
df_train.drop(labels = 'Arrival_Time', axis = 1, inplace = True)


In [0]:
#Cleaning Departure and Arrival Times
# Test Set

df_test['Depart_Time_Hour']=pd.to_datetime(df_test.Dep_Time).dt.hour
df_test['Depart_Time_Minutes']=pd.to_datetime(df_test.Dep_Time).dt.minute

df_test.drop(labels='Dep_Time', axis=1,inplace=True)

df_test['Arr_Time_Hour']=pd.to_datetime(df_test.Arrival_Time).dt.hour
df_test['Arr_Time_Minutes']=pd.to_datetime(df_test.Arrival_Time).dt.minute
df_test.drop(labels='Arrival_Time', axis=1,inplace=True)


In [0]:
#split input and target variables

X=df_train.drop(["Price"], axis=1)
y=np.log1p(df_train["Price"])

# y = df_train["Price"]
X_test = df_test

In [0]:
#Separate categorical and numerical columns in dataframe
X_categorical=X.select_dtypes(exclude=['int', 'float'])
X_numerical = X.select_dtypes(include=['int', 'float'])

X_test_categorical = X_test.select_dtypes(exclude=['int', 'float'])
X_test_numerical = X_test.select_dtypes(include=['int', 'float'])


In [0]:
X_numerical.head()
# Check if categorical columns are assigned properly
X_categorical.head()

#Create a back up copy specifically for plotting graphs
X_numerical_graph = X_numerical.copy()
X_categorical_graph = X_categorical.copy()

In [58]:
#We replace values based on insights gathered from graphs generated in further steps
X_categorical["Airline"].replace(to_replace={'Multiple carriers Premium economy':'Other', 
                                                        'Jet Airways Business':'Other',
                                                        'Vistara Premium economy':'Other',
                                                        'Trujet':'Other'
                                                   },    
                                        inplace=True)

X_test_categorical["Airline"].replace(to_replace={'Multiple carriers Premium economy':'Other', 
                                                        'Jet Airways Business':'Other',
                                                        'Vistara Premium economy':'Other',
                                                        'Trujet':'Other'
                                                   },    
                                        inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [59]:
#We replace values based on insights gathered from graphs generated in further steps
X_categorical["Additional_Info"].replace(to_replace={'Change airports':'Other', 
                                                        'Business class':'Other',
                                                        '1 Short layover':'Other',
                                                        'Red-eye flight':'Other',
                                                        '2 Long layover':'Other',   
                                                   },    
                                        inplace=True)
#We replace values based on insights gathered from graphs generated in further steps
X_test_categorical["Additional_Info"].replace(to_replace={'Change airports':'Other', 
                                                        'Business class':'Other',
                                                        '1 Short layover':'Other',
                                                        'Red-eye flight':'Other',
                                                        '2 Long layover':'Other',   
                                                   },    
                                        inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [0]:
#Label encode and hot encode categorical columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_categorical=X_categorical.apply(LabelEncoder().fit_transform)
X_test_categorical=X_test_categorical.apply(LabelEncoder().fit_transform)

In [61]:
#Check values after label encoding
X_categorical.head()

,Airline,Source,Destination,Route,Additional_Info,Day_of_week
0,3,0,5,18,3,3
1,1,3,0,84,3,6
2,4,2,1,118,3,3
3,3,3,0,91,3,3
4,3,0,5,29,3,0


In [62]:
#Check values after label encoding
X_numerical.head()

,Total_Stops,isWeekend,Day_of_journey,Month_of_journey,Duration_minutes,Depart_Time_Hour,Depart_Time_Minutes,Arr_Time_Hour,Arr_Time_Minutes
0,0,1,24,3,170,22,20,1,10
1,2,0,1,5,445,5,50,13,15
2,2,1,9,6,1140,9,25,4,25
3,1,1,12,5,325,18,5,23,30
4,1,0,1,3,285,16,50,21,35
